<a href="https://colab.research.google.com/github/alicewoo0925/miRNA-COVID19detection/blob/main/RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set Up

In [ ]:
# import modules
import os
from google.colab import files
from google.colab import drive
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split # to split the dataset into train and test
from sklearn.model_selection import StratifiedKFold # stratified k-fold
from sklearn import ensemble # random forest
from sklearn.decomposition import PCA

from sklearn.metrics import classification_report, confusion_matrix

# Train and Evaluation

In [ ]:
def getResult(y_test, prediction):
  print(classification_report(y_test, prediction)) # classification report

  #confusion matrix
  CM = confusion_matrix(y_test, prediction)
  CM = pd.DataFrame(confusion_matrix(y_test, prediction))
  CM.set_axis(['Mild','Moderate','Negative Control','Severe','Severe Negative'], axis=1, inplace=True)
  CM.set_axis(['Mild','Moderate','Negative Control','Severe','Severe Negative'], axis=0, inplace=True)
  print(CM)

## Full Dataset

In [ ]:
# split the dataset
X_train, X_test, Y_train, Y_test = train_test_split(
    df_clean.loc[:, df_clean.columns != 'Target'], # variables X include every columns except Target
    df_clean['Target'], # Y is target column
    test_size = 0.2) # 80% for train and 20% for test

In [ ]:
# random forest with full dataset
learner = ensemble.RandomForestClassifier()
model = learner.fit(X_train,Y_train)
prediction = model.predict(X_test)
getResult(Y_test,prediction)

## Selected Features by Random Forest

## SMOTE Data